In [65]:
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import lightgbm as lgb
from sklearn.metrics import f1_score
import numpy as np
from keras.models import load_model
CATEGORY = ['ecfg', 'flbmk', 'flg_3dsmk', 'insfg', 'ovrlt', 'scity', 'csmcu', 'cano', 'mchno', 'hcefg', 'bacno', 'contp', 'etymd', 'acqic']


def lgb_f1_score(y_true, y_pred):
    y_hat = np.round(y_pred)
    return 'f1', f1_score(y_true, y_hat), True

def label_encoder(x_train, x_test, df_test):
    from sklearn import preprocessing

    df = pd.concat([x_train,x_test,df_test], axis = 0)
    assert len(df)== len(x_train)+len(x_test)+len(df_test), "it should be same"
    
    for cat in CATEGORY:
        le = preprocessing.LabelEncoder()
        le.fit(df[cat].tolist())

        x_train[cat] = le.transform(x_train[cat].tolist()) 
        x_test[cat] = le.transform(x_test[cat].tolist()) 
        df_test[cat] = le.transform(df_test[cat].tolist()) 

    print ("*"* 100)
    print ("finished label encoding")
    return x_train,x_test,df_test

def pre_processing_for_auto_encoder(df):
    df = df.drop(['txkey'], axis=1)
    return df

def normalizing_for_auto_encoder(x_train,x_test,df_test):
    """
    return array
    """
    from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler

    df = pd.concat([x_train,x_test,df_test], axis = 0)
    assert len(df)== len(x_train)+len(x_test)+len(df_test), "it should be same"

    scaler = MinMaxScaler()
    
    scaler.fit(df)

    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    df_test = scaler.transform(df_test)
    print ("*"* 100)
    print ("finished data normalizing")
    return x_train,x_test,df_test

def add_auto_encoder_feature(df_raw,df):
    predictions = autoencoder.predict(df) # get reconstructed vector, 2-D, [num_samples, num_features]
    mse = np.mean(np.power(df - predictions, 2), axis=1) # get reconstructed error, 1-D, [num_samples,]
    df = pd.DataFrame(predictions, columns=["reconstructed_dim_{}".format(i) for i in range(predictions.shape[1])])
    df["reconstruction_error"] = mse
    out = pd.concat([df_raw.reset_index(), df.reset_index()], axis = 1)
    assert len(out)==len(df_raw)==len(df), "it should be same"
    print ("*"* 100)
    print ("finished adding auto_encoder_feature")    
    return out


In [23]:
args = {
 "train_file":"/data/yunrui_li/fraud/dataset/train.csv",
 "test_file": "/data/yunrui_li/fraud/dataset/test.csv"
}
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self
args = AttrDict(args)

In [9]:
#-------------------------
# load dataset
#-------------------------
df_train = pd.read_csv(args.train_file)
df_test = pd.read_csv(args.test_file)

for cat in CATEGORY:
    df_train[cat] = df_train[cat].astype('category')#.cat.codes
    df_test[cat] = df_test[cat].astype('category')

y_train = df_train['fraud_ind']
x_train = df_train.drop('fraud_ind', axis=1)

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)


In [68]:
#-------------------------
# auto_encoer
#-------------------------
x_train_, x_test_, df_test_ =  x_train.copy(), x_test.copy(), df_test.copy()

x_train_, x_test_, df_test_ = label_encoder(x_train_, x_test_, df_test_)

x_train_ = pre_processing_for_auto_encoder(x_train_) 
x_test_ = pre_processing_for_auto_encoder(x_test_)
df_test_ = pre_processing_for_auto_encoder(df_test_)

x_train_, x_test_, df_test_ = normalizing_for_auto_encoder(x_train_, x_test_, df_test_)



****************************************************************************************************
finished label encoding
****************************************************************************************************
finished data normalizing


In [69]:
autoencoder = load_model('/data/yunrui_li/fraud/fraud_detection/models/model.h5')

In [70]:
x_train = add_auto_encoder_feature(x_train, x_train_)
x_test = add_auto_encoder_feature(x_test,x_test_)
df_test = add_auto_encoder_feature(df_test,df_test_)


****************************************************************************************************
finished adding auto_encoder_feature
****************************************************************************************************
finished adding auto_encoder_feature
****************************************************************************************************
finished adding auto_encoder_feature


In [71]:
x_train

,index,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,index,reconstructed_dim_0,reconstructed_dim_1,reconstructed_dim_2,reconstructed_dim_3,reconstructed_dim_4,reconstructed_dim_5,reconstructed_dim_6,reconstructed_dim_7,reconstructed_dim_8,reconstructed_dim_9,reconstructed_dim_10,reconstructed_dim_11,reconstructed_dim_12,reconstructed_dim_13,reconstructed_dim_14,reconstructed_dim_15,reconstructed_dim_16,reconstructed_dim_17,reconstructed_dim_18,reconstructed_dim_19,reconstructed_dim_20,reconstruction_error
0,246363,6716,70705,111441,1135.92,5,62,N,5,N,N,5,N,0,79,154847.0,247,70305,N,5813,102,0,1096121,0,0.922855,0.424791,0.549933,0.098942,0.802798,0.838150,0.000000,0.474100,0.0,0.0,0.534933,0.0,0.0,0.643321,0.725546,0.596005,0.669012,0.0,0.862285,0.950920,0.0,0.000846
1,1125098,5975,157315,182130,407.87,5,62,N,5,N,N,5,N,0,60,85734.0,263,92655,N,5817,102,0,1155340,1,0.836729,0.947282,0.883448,0.091057,0.778249,0.811184,0.000000,0.482356,0.0,0.0,0.510775,0.0,0.0,0.408361,0.576736,0.618658,0.897041,0.0,0.913469,0.925958,0.0,0.003152
2,899275,6413,106996,95162,592.25,5,62,N,2,N,N,5,N,0,22,145031.0,432,51387,N,5817,102,0,1703412,2,0.948547,0.647410,0.484338,0.101923,0.786149,0.740434,0.000000,0.131991,0.0,0.0,0.505655,0.0,0.0,0.199399,0.537068,0.674349,0.474580,0.0,0.831086,1.031928,0.0,0.005075
3,367068,5720,92041,177639,387.00,5,35,Y,8,N,N,5,N,0,53,90729.0,192,67459,N,6221,75,0,727442,3,0.783295,0.552764,0.875027,0.086367,0.824696,0.830714,0.961113,0.737349,0.0,0.0,0.550814,0.0,0.0,0.336500,0.577321,0.561406,0.650205,0.0,0.715640,0.720010,0.0,0.012325
4,401687,3348,54901,165434,1016.34,5,62,Y,8,N,N,5,N,0,16,22104.0,289,54828,N,3460,46,0,665781,4,0.534770,0.317893,0.808989,0.072265,0.854872,0.697914,1.000768,0.909359,0.0,0.0,0.520517,0.0,0.0,0.000000,0.355243,0.577320,0.563246,0.0,0.483750,0.468888,0.0,0.006188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217424,1282388,6881,67997,204496,513.80,5,0,N,0,N,N,5,N,0,45,190409.0,457,59333,N,0,102,0,529978,1217424,1.028681,0.411723,0.989522,0.072657,0.815654,0.041676,0.000000,0.000000,0.0,0.0,0.553140,0.0,0.0,0.426053,0.691503,0.973342,0.549823,0.0,0.000000,0.943896,0.0,0.001026
1217425,1280825,5666,27859,112208,1201.74,5,62,N,2,N,N,5,N,0,60,224407.0,343,53687,N,5824,102,0,739724,1217425,0.963878,0.168792,0.564643,0.100854,0.803854,0.770255,0.000000,0.232540,0.0,0.0,0.520199,0.0,0.0,0.602829,0.728441,0.641203,0.485972,0.0,0.801012,1.013693,0.0,0.005498
1217426,1232733,6678,112297,191126,544.32,5,62,N,5,N,N,5,N,0,72,165017.0,247,23658,N,3454,102,0,1591467,1217426,0.925965,0.678926,0.917494,0.096374,0.833090,0.776348,0.000000,0.389214,0.0,0.0,0.539277,0.0,0.0,0.602720,0.771232,0.609791,0.221057,0.0,0.664241,0.873440,0.0,0.002620
1217427,1467373,6189,156598,129093,513.80,5,62,N,4,N,N,5,N,0,41,173110.0,263,92592,N,5817,102,0,1144115,1217427,0.916396,0.948923,0.643624,0.095868,0.782237,0.823418,0.000000,0.375980,0.0,0.0,0.541139,0.0,0.0,0.413250,0.604042,0.614242,0.863875,0.0,0.848603,0.946504,0.0,0.001521


In [72]:
x_test

,index,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,index,reconstructed_dim_0,reconstructed_dim_1,reconstructed_dim_2,reconstructed_dim_3,reconstructed_dim_4,reconstructed_dim_5,reconstructed_dim_6,reconstructed_dim_7,reconstructed_dim_8,reconstructed_dim_9,reconstructed_dim_10,reconstructed_dim_11,reconstructed_dim_12,reconstructed_dim_13,reconstructed_dim_14,reconstructed_dim_15,reconstructed_dim_16,reconstructed_dim_17,reconstructed_dim_18,reconstructed_dim_19,reconstructed_dim_20,reconstruction_error
0,622266,3348,2516,6013,0.00,5,62,Y,8,N,N,5,N,0,34,20435.0,380,54238,N,6580,38,0,696361,0,0.665554,0.001481,0.077520,0.089318,0.840319,0.800283,0.953866,0.923955,0.0,0.0,0.528360,0.0,0.0,0.043218,0.402739,0.556901,0.582350,0.0,0.719248,0.641312,0.0,0.021359
1,627835,6779,141670,24775,749.57,5,62,N,5,N,N,5,N,0,61,102251.0,263,7830,N,5817,102,0,1298622,1,0.923847,0.855973,0.142946,0.110215,0.823267,0.811254,0.000000,0.491574,0.0,0.0,0.506166,0.0,0.0,0.411962,0.668884,0.613479,0.094783,0.0,0.881883,0.968132,0.0,0.003526
2,588717,6291,17726,40077,420.41,5,62,N,5,N,N,5,N,0,87,202457.0,247,44598,N,135,102,0,1386622,2,0.868648,0.098731,0.199016,0.091208,0.869264,0.659758,0.000000,0.265779,0.0,0.0,0.584903,0.0,0.0,0.784111,0.828945,0.610466,0.409678,0.0,0.251413,0.709297,0.0,0.009876
3,1459196,3191,1883,113100,360.02,5,26,N,9,N,N,5,N,0,34,184433.0,247,72382,N,4711,52,0,697905,3,0.370798,0.010827,0.555925,0.068715,0.855330,0.524570,0.000000,0.801335,0.0,0.0,0.424586,0.0,0.0,0.433116,0.498000,0.651676,0.654123,0.0,0.475100,0.529213,0.0,0.011616
4,557187,6032,132444,78090,454.42,5,62,Y,2,N,N,5,N,0,81,105410.0,248,86284,N,5817,102,0,410144,4,1.034985,0.800152,0.416961,0.100152,0.801480,0.890742,0.961313,0.369160,0.0,0.0,0.614795,0.0,0.0,0.527389,0.702626,0.565440,0.848449,0.0,0.712590,0.904299,0.0,0.008680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304353,1117620,5975,54365,199156,298.10,5,62,N,5,N,N,1,N,0,59,150509.0,193,98198,N,5817,102,0,1725841,304353,0.808986,0.328635,0.966673,0.088365,0.784006,0.798212,0.000000,0.517066,0.0,0.0,0.496810,0.0,0.0,0.525515,0.617777,0.619351,0.898234,0.0,0.923658,0.925399,0.0,0.009826
304354,913521,6215,52433,201911,652.61,5,62,Y,2,N,N,5,N,0,32,210956.0,380,54238,N,6580,38,0,1867064,304354,0.947021,0.320284,1.016807,0.091915,0.820406,0.845500,0.992675,0.460993,0.0,0.0,0.579747,0.0,0.0,0.330343,0.632066,0.578074,0.495252,0.0,0.682954,0.824928,0.0,0.025193
304355,667638,6769,158962,1256,503.84,5,62,N,5,N,N,5,N,0,17,140435.0,251,77991,N,5817,102,0,278628,304355,0.920657,0.960593,0.040367,0.103857,0.789189,0.839181,0.000000,0.437307,0.0,0.0,0.544982,0.0,0.0,0.189592,0.506824,0.605971,0.730321,0.0,0.872708,0.941197,0.0,0.001307
304356,874431,6881,121409,200678,513.80,5,0,N,0,N,N,5,N,0,28,212609.0,457,74337,N,0,102,0,509859,304356,1.054190,0.736275,0.977549,0.072051,0.807400,0.052642,0.000000,0.000000,0.0,0.0,0.575555,0.0,0.0,0.341891,0.652885,0.970422,0.681876,0.0,0.000000,0.932653,0.0,0.004061


In [73]:
df_test

,index,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,index,reconstructed_dim_0,reconstructed_dim_1,reconstructed_dim_2,reconstructed_dim_3,reconstructed_dim_4,reconstructed_dim_5,reconstructed_dim_6,reconstructed_dim_7,reconstructed_dim_8,reconstructed_dim_9,reconstructed_dim_10,reconstructed_dim_11,reconstructed_dim_12,reconstructed_dim_13,reconstructed_dim_14,reconstructed_dim_15,reconstructed_dim_16,reconstructed_dim_17,reconstructed_dim_18,reconstructed_dim_19,reconstructed_dim_20,reconstruction_error
0,0,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,102,215328.0,457,59360,N,0,102,0,592489,0,0.998929,0.975009,0.571391,0.078702,0.823315,0.022611,0.0,0.000000,0.0,0.0,0.531912,0.0,0.0,0.838975,0.865554,0.971973,0.545179,0.0,0.000000,0.975410,0.0,0.000331
1,1,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,102,222007.0,457,59360,N,0,102,0,592452,1,1.004439,0.974289,0.571477,0.078850,0.823989,0.024944,0.0,0.000000,0.0,0.0,0.534598,0.0,0.0,0.847371,0.871707,0.970435,0.542753,0.0,0.000000,0.974925,0.0,0.000460
2,2,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,100,170013.0,457,59360,N,0,102,0,590212,2,0.963251,0.980068,0.571004,0.077697,0.818546,0.007649,0.0,0.000000,0.0,0.0,0.514845,0.0,0.0,0.767955,0.818097,0.982271,0.561391,0.0,0.000000,0.977682,0.0,0.000914
3,3,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,100,165914.0,457,59360,N,0,102,0,590209,3,0.959910,0.980377,0.570878,0.077607,0.818137,0.006255,0.0,0.000000,0.0,0.0,0.513232,0.0,0.0,0.762495,0.814204,0.983195,0.562819,0.0,0.000000,0.977932,0.0,0.001098
4,4,6881,163188,116799,513.80,5,0,N,0,N,N,5,N,0,102,215311.0,457,59360,N,0,102,0,592488,4,0.998915,0.975011,0.571391,0.078701,0.823313,0.022605,0.0,0.000000,0.0,0.0,0.531906,0.0,0.0,0.838954,0.865539,0.971977,0.545185,0.0,0.000000,0.975411,0.0,0.000331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421660,421660,5975,147326,79511,633.76,5,62,N,2,N,N,5,N,0,116,111228.0,343,87282,N,5817,102,0,1187507,421660,0.895513,0.882780,0.404280,0.099306,0.780685,0.737841,0.0,0.221015,0.0,0.0,0.493855,0.0,0.0,0.805380,0.760082,0.660472,0.851869,0.0,0.860679,1.048190,0.0,0.006897
421661,421661,6716,14305,136493,952.84,5,62,N,5,N,N,8,N,0,107,85839.0,245,48784,N,5859,102,0,1182598,421661,0.946644,0.078175,0.654883,0.099570,0.813764,0.847746,0.0,0.487894,0.0,0.0,0.537481,0.0,0.0,0.706360,0.776216,0.588673,0.519854,0.0,0.853101,0.953096,0.0,0.016037
421662,421662,5975,156543,137963,713.42,5,62,N,4,N,N,5,N,0,111,184921.0,263,98326,N,5817,102,0,898724,421662,0.902520,0.933021,0.677375,0.095473,0.789746,0.815101,0.0,0.416336,0.0,0.0,0.530083,0.0,0.0,0.889535,0.810632,0.606638,0.927118,0.0,0.847653,0.966925,0.0,0.000496
421663,421663,6231,156543,137964,903.94,5,62,N,5,N,N,5,N,0,114,144434.0,251,17763,N,5817,102,0,971467,421663,0.905176,0.937840,0.667205,0.103916,0.825396,0.808218,0.0,0.526260,0.0,0.0,0.497733,0.0,0.0,0.828098,0.854909,0.606442,0.182871,0.0,0.874274,0.966052,0.0,0.003976


In [61]:
pd.options.display.max_columns = 100

pd.concat([x_train.reset_index(), df.reset_index()], axis = 1)

,index,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,index,reconstructed_dim_0,reconstructed_dim_1,reconstructed_dim_2,reconstructed_dim_3,reconstructed_dim_4,reconstructed_dim_5,reconstructed_dim_6,reconstructed_dim_7,reconstructed_dim_8,reconstructed_dim_9,reconstructed_dim_10,reconstructed_dim_11,reconstructed_dim_12,reconstructed_dim_13,reconstructed_dim_14,reconstructed_dim_15,reconstructed_dim_16,reconstructed_dim_17,reconstructed_dim_18,reconstructed_dim_19,reconstructed_dim_20,reconstruction_error
0,246363,6716,70705,111441,1135.92,5,62,N,5,N,N,5,N,0,79,154847.0,247,70305,N,5813,102,0,1096121,0,0.922855,0.424791,0.549933,0.098942,0.802798,0.838150,0.000000,0.474100,0.0,0.0,0.534933,0.0,0.0,0.643321,0.725546,0.596005,0.669012,0.0,0.862285,0.950920,0.0,0.000846
1,1125098,5975,157315,182130,407.87,5,62,N,5,N,N,5,N,0,60,85734.0,263,92655,N,5817,102,0,1155340,1,0.836729,0.947282,0.883448,0.091057,0.778249,0.811184,0.000000,0.482356,0.0,0.0,0.510775,0.0,0.0,0.408361,0.576736,0.618658,0.897041,0.0,0.913469,0.925958,0.0,0.003152
2,899275,6413,106996,95162,592.25,5,62,N,2,N,N,5,N,0,22,145031.0,432,51387,N,5817,102,0,1703412,2,0.948547,0.647410,0.484338,0.101923,0.786149,0.740434,0.000000,0.131991,0.0,0.0,0.505655,0.0,0.0,0.199399,0.537068,0.674349,0.474580,0.0,0.831086,1.031928,0.0,0.005075
3,367068,5720,92041,177639,387.00,5,35,Y,8,N,N,5,N,0,53,90729.0,192,67459,N,6221,75,0,727442,3,0.783295,0.552764,0.875027,0.086367,0.824696,0.830714,0.961113,0.737349,0.0,0.0,0.550814,0.0,0.0,0.336500,0.577321,0.561406,0.650205,0.0,0.715640,0.720010,0.0,0.012325
4,401687,3348,54901,165434,1016.34,5,62,Y,8,N,N,5,N,0,16,22104.0,289,54828,N,3460,46,0,665781,4,0.534770,0.317893,0.808989,0.072265,0.854872,0.697914,1.000768,0.909359,0.0,0.0,0.520517,0.0,0.0,0.000000,0.355243,0.577320,0.563246,0.0,0.483750,0.468888,0.0,0.006188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1217424,1282388,6881,67997,204496,513.80,5,0,N,0,N,N,5,N,0,45,190409.0,457,59333,N,0,102,0,529978,1217424,1.028681,0.411723,0.989522,0.072657,0.815654,0.041676,0.000000,0.000000,0.0,0.0,0.553140,0.0,0.0,0.426053,0.691503,0.973342,0.549823,0.0,0.000000,0.943896,0.0,0.001026
1217425,1280825,5666,27859,112208,1201.74,5,62,N,2,N,N,5,N,0,60,224407.0,343,53687,N,5824,102,0,739724,1217425,0.963878,0.168792,0.564643,0.100854,0.803854,0.770255,0.000000,0.232540,0.0,0.0,0.520199,0.0,0.0,0.602829,0.728441,0.641203,0.485972,0.0,0.801012,1.013693,0.0,0.005498
1217426,1232733,6678,112297,191126,544.32,5,62,N,5,N,N,5,N,0,72,165017.0,247,23658,N,3454,102,0,1591467,1217426,0.925965,0.678926,0.917494,0.096374,0.833090,0.776348,0.000000,0.389214,0.0,0.0,0.539277,0.0,0.0,0.602720,0.771232,0.609791,0.221057,0.0,0.664241,0.873440,0.0,0.002620
1217427,1467373,6189,156598,129093,513.80,5,62,N,4,N,N,5,N,0,41,173110.0,263,92592,N,5817,102,0,1144115,1217427,0.916396,0.948923,0.643624,0.095868,0.782237,0.823418,0.000000,0.375980,0.0,0.0,0.541139,0.0,0.0,0.413250,0.604042,0.614242,0.863875,0.0,0.848603,0.946504,0.0,0.001521


In [54]:
df.shape

(1217429, 22)

In [55]:
x_train.shape

(1217429, 22)

In [57]:
x_train

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
246363,6716,70705,111441,1135.92,5,62,N,5,N,N,5,N,0,79,154847.0,247,70305,N,5813,102,0,1096121
1125098,5975,157315,182130,407.87,5,62,N,5,N,N,5,N,0,60,85734.0,263,92655,N,5817,102,0,1155340
899275,6413,106996,95162,592.25,5,62,N,2,N,N,5,N,0,22,145031.0,432,51387,N,5817,102,0,1703412
367068,5720,92041,177639,387.00,5,35,Y,8,N,N,5,N,0,53,90729.0,192,67459,N,6221,75,0,727442
401687,3348,54901,165434,1016.34,5,62,Y,8,N,N,5,N,0,16,22104.0,289,54828,N,3460,46,0,665781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1282388,6881,67997,204496,513.80,5,0,N,0,N,N,5,N,0,45,190409.0,457,59333,N,0,102,0,529978
1280825,5666,27859,112208,1201.74,5,62,N,2,N,N,5,N,0,60,224407.0,343,53687,N,5824,102,0,739724
1232733,6678,112297,191126,544.32,5,62,N,5,N,N,5,N,0,72,165017.0,247,23658,N,3454,102,0,1591467
1467373,6189,156598,129093,513.80,5,62,N,4,N,N,5,N,0,41,173110.0,263,92592,N,5817,102,0,1144115
